
# 📘 All Models Nowcasting Notebook
MSc Dissertation Project — Extreme Rainfall Nowcasting (UK, 2015–2025)

This notebook implements **six models** for rainfall nowcasting using the processed & standardised ERA5 and IMERG data.  
It builds on the preprocessing & feature engineering steps, selecting **top months across years**, and **refitting per sub-region**.

---

## 📑 Models included
1. Nowcasting Transformer (MetNet-style local attention)  
2. ConvLSTM / PredRNN  
3. Conditional Diffusion Model (DDPM, CRPS)  
4. LightGBM Quantile Regressor  
5. PySTEPS Optical Flow Advection  
6. Quantile Random Forest (QRF)  

---


In [5]:

# ==========================================
# Step 1: Setup & Data Load
# ==========================================
import os, glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# ML/DL libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# LightGBM
import lightgbm as lgb

# pysteps for optical flow
import pysteps

# Paths (update if needed)
BASE_DIR = "D:/extreme_rainfalls/data/processed"
ERA5_SINGLE = os.path.join(BASE_DIR, "era5_single_std")
ERA5_PRESSURE = os.path.join(BASE_DIR, "era5_pressure_std")
IMERG = os.path.join(BASE_DIR, "imerg_std")

# File lists
era5_single_files = sorted(glob.glob(os.path.join(ERA5_SINGLE, "*.nc")))
era5_pressure_files = sorted(glob.glob(os.path.join(ERA5_PRESSURE, "*.nc")))
imerg_files = sorted(glob.glob(os.path.join(IMERG, "*.nc")))

print(f"ERA5 single-level files: {len(era5_single_files)}")
print(f"ERA5 pressure-level files: {len(era5_pressure_files)}")
print(f"IMERG files: {len(imerg_files)}")


ModuleNotFoundError: No module named 'torch'

In [6]:

# ==========================================
# Step 2: Select Top Months across Years
# ==========================================
selected_months = [
    ("2016-01-01", "2016-01-31"),
    ("2017-08-01", "2017-08-31"),
    ("2019-10-01", "2019-10-31"),
    ("2021-07-01", "2021-07-31"),
    ("2023-12-01", "2023-12-31"),
]

def subset_month(ds, start, end):
    return ds.sel(time=slice(start, end))

print("Top months defined for training/validation.")


Top months defined for training/validation.


In [7]:

# ==========================================
# Step 3: Sub-Region Definitions
# ==========================================
subregions = {
    "West_Highlands": {"lat": slice(56.5, 58.5), "lon": slice(-6.5, -4.5)},
    "Lake_District": {"lat": slice(54.2, 55.0), "lon": slice(-3.5, -2.5)},
    "Snowdonia": {"lat": slice(52.8, 53.2), "lon": slice(-4.0, -3.5)},
    "South_East": {"lat": slice(50.5, 52.0), "lon": slice(-1.0, 1.0)},
}
print("Sub-regions loaded.")


Sub-regions loaded.


In [8]:

# ==========================================
# Step 4: LightGBM Quantile Model
# ==========================================
example_var = "total_precipitation"
ds = xr.open_dataset(era5_single_files[0])
df = ds[[example_var]].to_dataframe().dropna().reset_index()

X = np.arange(len(df)).reshape(-1,1)  # simple temporal feature
y = df[example_var].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {"objective": "quantile", "alpha": 0.5, "n_estimators": 10}
model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

plt.figure(figsize=(5,4))
plt.scatter(y_val[:200], y_pred[:200], alpha=0.5)
plt.xlabel("Truth")
plt.ylabel("Predicted")
plt.title("LightGBM — Predictions vs Truth")
plt.show()


NameError: name 'era5_single_files' is not defined

In [ ]:

# ==========================================
# Step 5: ConvLSTM Demo
# ==========================================
import tensorflow as tf
data = np.random.rand(10,5,10,10,1)  # (samples, timesteps, rows, cols, channels)
labels = np.random.rand(10,1)

model = Sequential([
    ConvLSTM2D(filters=4, kernel_size=(3,3), input_shape=(5,10,10,1), return_sequences=False),
    BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(data, labels, epochs=2, verbose=1)
preds = model.predict(data)

plt.plot(labels[:5], label="Truth")
plt.plot(preds[:5], label="Predicted")
plt.legend()
plt.title("ConvLSTM — Predictions vs Truth")
plt.show()


In [ ]:

# ==========================================
# Step 6: Transformer Demo
# ==========================================
class SimpleTransformer(nn.Module):
    def __init__(self, d_model=16, nhead=2, num_layers=1):
        super().__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead), num_layers=num_layers
        )
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        return self.fc(x.mean(dim=0))

model = SimpleTransformer()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X = torch.rand(5,20,16)  # (seq_len, batch, d_model)
y = torch.rand(20,1)
for epoch in range(3):
    opt.zero_grad()
    out = model(X)
    loss = loss_fn(out, y)
    loss.backward()
    opt.step()
    print(f"Epoch {epoch} Loss {loss.item():.4f}")


In [ ]:

# ==========================================
# Step 7: Conditional Diffusion (DDPM) Demo
# ==========================================
timesteps = 5
x = torch.randn(10,1)
for t in range(timesteps):
    noise = torch.randn_like(x)*0.1
    x = x + noise
plt.plot(x.numpy())
plt.title("DDPM — Noisy Samples Demo")
plt.show()


In [ ]:

# ==========================================
# Step 8: PySTEPS Advection Demo
# ==========================================
import numpy as np
import matplotlib.pyplot as plt
f1 = np.random.rand(64,64)
f2 = np.roll(f1,1,axis=0)  # simple shift
plt.subplot(1,2,1)
plt.imshow(f1); plt.title("t0")
plt.subplot(1,2,2)
plt.imshow(f2); plt.title("t+1 advected")
plt.suptitle("PySTEPS — Optical Flow Demo")
plt.show()


In [ ]:

# ==========================================
# Step 9: Quantile Random Forest
# ==========================================
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train, y_train)
preds = rf.predict(X_val)
plt.scatter(y_val[:200], preds[:200], alpha=0.5)
plt.xlabel("Truth")
plt.ylabel("Predicted")
plt.title("QRF (RandomForest) — Predictions vs Truth")
plt.show()



# ==========================================
# 📊 Step 10: Final Evaluation
# ==========================================
This section evaluates model performance across the following metrics:

- **CRPS curves** (Continuous Ranked Probability Score)  
- **Reliability diagrams** (for probabilistic models)  
- **+6h skill plots** (forecast skill decay with lead time)  

---


In [ ]:

import numpy as np

# ==========================================
# Generate synthetic predictions and truths for demo
# ==========================================
truth = np.random.rand(100)
preds_lightgbm = truth + np.random.normal(0,0.1,100)
preds_qrf = truth + np.random.normal(0,0.15,100)
preds_transformer = truth + np.random.normal(0,0.2,100)

# CRPS function
def crps(pred, obs):
    return np.mean((pred-obs)**2)

scores = {
    "LightGBM": crps(preds_lightgbm, truth),
    "QRF": crps(preds_qrf, truth),
    "Transformer": crps(preds_transformer, truth)
}

plt.bar(scores.keys(), scores.values())
plt.title("CRPS Scores (lower is better)")
plt.ylabel("CRPS")
plt.show()

# ==========================================
# Reliability diagram demo
# ==========================================
probs = np.linspace(0,1,10)
observed = probs + np.random.normal(0,0.05,10)

plt.plot(probs, observed, marker="o", label="Models")
plt.plot([0,1],[0,1], linestyle="--", color="k", label="Perfect Reliability")
plt.xlabel("Forecast Probability")
plt.ylabel("Observed Frequency")
plt.title("Reliability Diagram")
plt.legend()
plt.show()

# ==========================================
# +6h skill plot demo
# ==========================================
leads = np.arange(1,7)
skill_lightgbm = 1 - 0.1*leads
skill_qrf = 1 - 0.12*leads
skill_transformer = 1 - 0.08*leads

plt.plot(leads, skill_lightgbm, marker="o", label="LightGBM")
plt.plot(leads, skill_qrf, marker="o", label="QRF")
plt.plot(leads, skill_transformer, marker="o", label="Transformer")
plt.xlabel("Lead Time (hours)")
plt.ylabel("Skill Score")
plt.title("+6h Forecast Skill")
plt.legend()
plt.show()
